In [39]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import os

In [40]:
# Make torch deterministc
_ = torch.manual_seed(0)

In [41]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Load the MNIST dataset
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# Create a dataloader for the training
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=10, shuffle=True)

# Load the MNIST test set
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=10, shuffle=True)

# Define the device
device = "cuda"

In [42]:
class VerySimpleNet(nn.Module):
    def __init__(self, hidden_size_1=100, hidden_size_2=100):
        super(VerySimpleNet,self).__init__()
        self.linear1 = nn.Linear(28*28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [43]:
net = VerySimpleNet().to(device)

In [44]:
def train(train_loader, net, epochs=5, total_iterations_limit=None):
    cross_el = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

    total_iterations = 0

    for epoch in range(epochs):
        net.train()

        loss_sum = 0
        num_iterations = 0

        data_iterator = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        if total_iterations_limit is not None:
            data_iterator.total = total_iterations_limit
        for data in data_iterator:
            num_iterations += 1
            total_iterations += 1
            x, y = data
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = net(x.view(-1, 28*28))
            loss = cross_el(output, y)
            loss_sum += loss.item()
            avg_loss = loss_sum / num_iterations
            data_iterator.set_postfix(loss=avg_loss)
            loss.backward()
            optimizer.step()

            if total_iterations_limit is not None and total_iterations >= total_iterations_limit:
                return
            
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp_delme.p")
    print('Size (KB):', os.path.getsize("temp_delme.p")/1e3)
    os.remove('temp_delme.p')

MODEL_FILENAME = 'simplenet_ptq.pt'

if Path(MODEL_FILENAME).exists():
    net.load_state_dict(torch.load(MODEL_FILENAME))
    print('Loaded model from disk')
else:
    train(train_loader, net, epochs=1)
    # Save the model to disk
    torch.save(net.state_dict(), MODEL_FILENAME)

Epoch 1: 100%|██████████| 6000/6000 [00:35<00:00, 169.94it/s, loss=0.223]


In [45]:
def test(model: nn.Module, total_iterations: int = None):
    correct = 0
    total = 0
    
    iterations = 0
    
    model.eval()
    
    with torch.no_grad():
        for data in tqdm(test_loader, desc='Testing'):
            x, y = data
            x = x.to(device)
            y = y.to(device)
            output = model(x.view(-1, 784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total += 1
            iterations += 1
            if total_iterations is not None and iterations >= total_iterations:
                break
    print(f"Accuracy {round(correct/total, 3)}")

In [46]:
print("Weights before quantization")
print(net.linear1.weight)
print(net.linear1.weight.dtype)

Weights before quantization
Parameter containing:
tensor([[ 0.0046,  0.0240, -0.0245,  ...,  0.0268,  0.0086,  0.0069],
        [-0.0157, -0.0109, -0.0064,  ..., -0.0162, -0.0019, -0.0259],
        [ 0.0181,  0.0532,  0.0050,  ...,  0.0179,  0.0395,  0.0463],
        ...,
        [ 0.0234,  0.0271, -0.0076,  ..., -0.0129,  0.0063, -0.0306],
        [-0.0115, -0.0034,  0.0287,  ...,  0.0090,  0.0090,  0.0085],
        [ 0.0202,  0.0149,  0.0007,  ...,  0.0371, -0.0122,  0.0080]],
       device='cuda:0', requires_grad=True)
torch.float32


In [47]:
print("Size of the model before quantization")
print_size_of_model(net)

Size of the model before quantization
Size (KB): 361.062


In [48]:
print("Accuracy of the model before quantization")
test(net)

Accuracy of the model before quantization


Testing: 100%|██████████| 1000/1000 [00:02<00:00, 389.70it/s]

Accuracy 0.963


## Insert min-max observes in the model

In [49]:
class QuantizedVerySimpleNet(nn.Module):
    def __init__(self, hidden_size_1=100, hidden_size_2=100):
        super(QuantizedVerySimpleNet,self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.linear1 = nn.Linear(28*28, hidden_size_1) 
        self.linear2 = nn.Linear(hidden_size_1, hidden_size_2) 
        self.linear3 = nn.Linear(hidden_size_2, 10)
        self.relu = nn.ReLU()
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, img):
        x = img.view(-1, 28*28)
        x = self.quant(x)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        x = self.dequant(x)
        return x

In [50]:
net_quantized = QuantizedVerySimpleNet().to(device)
# Copy weights from unquantized model
net_quantized.load_state_dict(net.state_dict())
net_quantized.eval()

net_quantized.qconfig = torch.ao.quantization.default_qconfig
net_quantized = torch.ao.quantization.prepare(net_quantized) # Insert observers
net_quantized

QuantizedVerySimpleNet(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear1): Linear(
    in_features=784, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear2): Linear(
    in_features=100, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (linear3): Linear(
    in_features=100, out_features=10, bias=True
    (activation_post_process): MinMaxObserver(min_val=inf, max_val=-inf)
  )
  (relu): ReLU()
  (dequant): DeQuantStub()
)

## Calibre the model using the test set

In [51]:
test(net_quantized)

Testing: 100%|██████████| 1000/1000 [00:03<00:00, 317.00it/s]

Accuracy 0.963


In [52]:
print("Check statistics of the various layers")
net_quantized

Check statistics of the various layers


QuantizedVerySimpleNet(
  (quant): QuantStub(
    (activation_post_process): MinMaxObserver(min_val=-0.4242129623889923, max_val=2.821486711502075)
  )
  (linear1): Linear(
    in_features=784, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=-53.876556396484375, max_val=38.96775436401367)
  )
  (linear2): Linear(
    in_features=100, out_features=100, bias=True
    (activation_post_process): MinMaxObserver(min_val=-26.389554977416992, max_val=26.55878448486328)
  )
  (linear3): Linear(
    in_features=100, out_features=10, bias=True
    (activation_post_process): MinMaxObserver(min_val=-27.63741683959961, max_val=23.74498748779297)
  )
  (relu): ReLU()
  (dequant): DeQuantStub()
)

## Quantize the model using the statistics collected

In [53]:
net_quantized = torch.ao.quantization.convert(net_quantized)
print("Statistics of various layers")
net_quantized

Statistics of various layers


QuantizedVerySimpleNet(
  (quant): Quantize(scale=tensor([0.0256], device='cuda:0'), zero_point=tensor([17], device='cuda:0'), dtype=torch.quint8)
  (linear1): QuantizedLinear(in_features=784, out_features=100, scale=0.7310575842857361, zero_point=74, qscheme=torch.per_tensor_affine)
  (linear2): QuantizedLinear(in_features=100, out_features=100, scale=0.4169160723686218, zero_point=63, qscheme=torch.per_tensor_affine)
  (linear3): QuantizedLinear(in_features=100, out_features=10, scale=0.4045858681201935, zero_point=68, qscheme=torch.per_tensor_affine)
  (relu): ReLU()
  (dequant): DeQuantize()
)

## Print weights of the model after quantization

In [54]:
print("Weights after quantization")
print(torch.int_repr(net_quantized.linear1.weight()))

Weights after quantization
tensor([[ 1,  5, -5,  ...,  6,  2,  2],
        [-4, -2, -1,  ..., -4,  0, -6],
        [ 4, 12,  1,  ...,  4,  9, 10],
        ...,
        [ 5,  6, -2,  ..., -3,  1, -7],
        [-3, -1,  6,  ...,  2,  2,  2],
        [ 4,  3,  0,  ...,  8, -3,  2]], device='cuda:0', dtype=torch.int8)


In [55]:
print("Original weights:")
print(net.linear1.weight)
print('')
print("Dequentaized weights:")
print(torch.dequantize(net_quantized.linear1.weight()))
print('')

Original weights:
Parameter containing:
tensor([[ 0.0046,  0.0240, -0.0245,  ...,  0.0268,  0.0086,  0.0069],
        [-0.0157, -0.0109, -0.0064,  ..., -0.0162, -0.0019, -0.0259],
        [ 0.0181,  0.0532,  0.0050,  ...,  0.0179,  0.0395,  0.0463],
        ...,
        [ 0.0234,  0.0271, -0.0076,  ..., -0.0129,  0.0063, -0.0306],
        [-0.0115, -0.0034,  0.0287,  ...,  0.0090,  0.0090,  0.0085],
        [ 0.0202,  0.0149,  0.0007,  ...,  0.0371, -0.0122,  0.0080]],
       device='cuda:0', requires_grad=True)

Dequentaized weights:
tensor([[ 0.0045,  0.0225, -0.0225,  ...,  0.0270,  0.0090,  0.0090],
        [-0.0180, -0.0090, -0.0045,  ..., -0.0180,  0.0000, -0.0270],
        [ 0.0180,  0.0539,  0.0045,  ...,  0.0180,  0.0404,  0.0449],
        ...,
        [ 0.0225,  0.0270, -0.0090,  ..., -0.0135,  0.0045, -0.0314],
        [-0.0135, -0.0045,  0.0270,  ...,  0.0090,  0.0090,  0.0090],
        [ 0.0180,  0.0135,  0.0000,  ...,  0.0359, -0.0135,  0.0090]],
       device='cuda:0')



In [56]:
print("Size of the model after quantization")
print_size_of_model(net_quantized)

Size of the model after quantization
Size (KB): 95.458


In [58]:
# print("Testing the model after quantization")
# test(net_quantized)